In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#지우지말것
#데이터 저장 2000 크롤링해서 txt파일로

import requests
import pandas as pd
from bs4 import BeautifulSoup

pd.set_option('display.max_columns', None)

def crawl_player_data(year, start_num=0, end_num=999):
    base_url = "https://www.kleague.com/record/playerDetail.do?playerId={}"
    player_id_list = []

    for num in range(start_num, end_num + 1):
        player_id = f"{year}{num:04d}"
        url = base_url.format(player_id)
        response = requests.get(url)

        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            error_message = soup.find(string="존재하지 않는 주소를 입력하셨거나")

            if not error_message:
                player_id_list.append(player_id)

    return player_id_list

# playerId 리스트 크롤링
for n in range(13,14):
    # playerId 리스트 크롤링
    player_id_list = crawl_player_data(2010 + n)

    # playerId 리스트를 파일에 저장
    file_path = f'/content/drive/MyDrive/project_chatbot/player_id_list{2010+str(n)}.txt'
    with open(file_path, "w") as file:
        for player_id in player_id_list:
            file.write(player_id + "\n")

    print(f"Player ID list for {2010+str(n)} saved successfully!")

    #굿되네

In [ ]:
#txt->excel로
import requests
import pandas as pd
from bs4 import BeautifulSoup

# 파일에서 playerId 목록을 읽어오는 함수
def read_player_ids(file_path):
    with open(file_path, 'r') as file:
        player_ids = [line.strip() for line in file.readlines()]
    return player_ids

# 크롤링 및 데이터프레임 생성 함수
def create_dataframe(playerId_list):
    dfs = []  # 여러 개의 데이터프레임을 저장할 리스트

    for playerId in playerId_list:
        url = f'https://www.kleague.com/record/playerDetail.do?playerId={playerId}'

        # HTTP GET 요청 보내기
        response = requests.get(url)

        # 응답의 상태코드 확인
        if response.status_code == 200:
            # 응답 내용을 BeautifulSoup 객체로 파싱
            soup = BeautifulSoup(response.text, 'html.parser')

            # 클래스가 "table-wrap"인 요소를 찾기
            table_wrap = soup.find(class_="table-wrap")

            # 클래스가 "style2 center"인 요소를 찾기
            style2_center = soup.find(class_='style2 center')

            # 테이블 요소를 DataFrame으로 변환
            df_style = pd.read_html(str(style2_center))[0]
            df_wrap = pd.read_html(str(table_wrap))[0]

            # 등록번호 추가
            df_wrap['등록번호'] = playerId

            # 테이블 조작 및 열 순서 변경
            df_wrap['이름'] = df_style.iloc[0, 1]
            df_wrap['포지션'] = df_style.iloc[1, 3]
            df_wrap['키'] = df_style.iloc[3, 1]
            df_wrap['몸무게'] = df_style.iloc[3, 3]

            new_order = ['등록번호', '이름', '키', '몸무게', '포지션'] + [col for col in df_wrap.columns if
                                                       col not in ['등록번호', '이름', '키', '몸무게', '포지션']]
            df = df_wrap[new_order]
            dfs.append(df)
        else:
            print("Error:", response.status_code)

    # 모든 데이터프레임을 결합
    result_df = pd.concat(dfs, ignore_index=True)
    return result_df

# 파일 경로
file_base_path = '/content/drive/MyDrive/project_chatbot/13_23/player_id_list'
excel_base_path = '/content/drive/MyDrive/project_chatbot/13_23/player_data_'

# player_id_list2013.txt부터 player_id_list2023.txt까지 반복하며 처리
for year in range(2013, 2024):
    file_path = f"{file_base_path}{year}.txt"
    excel_file_path = f"{excel_base_path}{year}.xlsx"

    # 파일에서 playerId 목록 읽어오기
    playerId_list = read_player_ids(file_path)

    # 데이터프레임 생성
    result_df = create_dataframe(playerId_list)

    try:
        # 불필요한 열 제거 및 결측치 처리
        result_df.drop(columns=['구단', '재임년도', '승', '무', '패', '비고'], inplace=True)
    except KeyError:
        # 만약 열이 존재하지 않으면 그냥 넘어감
        pass

    result_df.dropna(inplace=True)
    result_df[['연도', '출장', '득점', '도움', '골킥', '코너킥', '오프사이드', '슈팅', '파울', '실점', '경고', '퇴장']] = result_df[
        ['연도', '출장', '득점', '도움', '골킥', '코너킥', '오프사이드', '슈팅', '파울', '실점', '경고', '퇴장']].astype(int)

    # 데이터프레임을 엑셀 파일로 저장
    result_df.to_excel(excel_file_path, index=False)

    print(f"{year}년 데이터가 성공적으로 엑셀 파일로 저장되었습니다.")


In [15]:
import pandas as pd

player2013 = pd.read_excel('/content/drive/MyDrive/hjh_kita_directory/hjh2차챗봇프로젝트/playerdata/player_data_2013.xlsx')
player2013

,등록번호,이름,키,몸무게,포지션,연도,리그,소속,출장,득점,도움,골킥,코너킥,오프사이드,슈팅,파울,실점,경고,퇴장
0,20130026,배천석,186,78,FW,2017,K리그1,전남,8,0,1,0,0,1,3,7,0,0,0
1,20130026,배천석,186,78,FW,2016,K리그1,전남,23,3,3,0,0,4,13,12,0,3,0
2,20130026,배천석,186,78,FW,2015,K리그1,부산,21,1,1,0,0,4,30,36,0,0,0
3,20130026,배천석,186,78,FW,2014,K리그1,포항,4,0,0,0,0,1,0,5,0,0,0
4,20130026,배천석,186,78,FW,2013,K리그1,포항,20,4,2,0,0,8,19,19,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
476,20130434,호드리고,174,70,FW,2013,K리그1,제주,3,0,0,0,0,0,2,2,0,1,0
477,20130435,티아고,172,68,MF,2013,K리그1,전북,14,1,2,0,25,3,13,4,0,0,0
478,20130436,아리아스,175,66,FW,2013,K리그1,대전,15,6,0,0,9,11,34,37,0,3,0
479,20130439,기가,177,75,FW,2014,K리그1,성남,1,0,0,0,0,0,0,0,0,0,0


In [16]:
player2013.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 481 entries, 0 to 480
Data columns (total 19 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   등록번호    481 non-null    int64 
 1   이름      481 non-null    object
 2   키       481 non-null    object
 3   몸무게     481 non-null    object
 4   포지션     481 non-null    object
 5   연도      481 non-null    int64 
 6   리그      481 non-null    object
 7   소속      481 non-null    object
 8   출장      481 non-null    int64 
 9   득점      481 non-null    int64 
 10  도움      481 non-null    int64 
 11  골킥      481 non-null    int64 
 12  코너킥     481 non-null    int64 
 13  오프사이드   481 non-null    int64 
 14  슈팅      481 non-null    int64 
 15  파울      481 non-null    int64 
 16  실점      481 non-null    int64 
 17  경고      481 non-null    int64 
 18  퇴장      481 non-null    int64 
dtypes: int64(13), object(6)
memory usage: 71.5+ KB


In [9]:
import pandas as pd
import os

# 파일 경로 설정
directory = '/content/drive/MyDrive/hjh_kita_directory/hjh2차챗봇프로젝트/선수데이터셋'
start_year = 2013
end_year = 2023

# 시작 연도부터 끝 연도까지 반복
for year in range(start_year, end_year + 1):
    # Excel 파일 경로
    file_path = os.path.join(directory, f'player_data_{year}.xlsx')

    # 파일이 존재하는지 확인
    if os.path.exists(file_path):
        # Excel 파일 읽기
        df = pd.read_excel(file_path)

        # 널 값이 포함된 행 제거
        df_cleaned = df.dropna()

        # 변경 사항을 같은 파일에 저장
        df_cleaned.to_excel(file_path, index=False)

        print(f'{year}년 데이터에서 널 값이 제거되었습니다.')
    else:
        print(f'{year}년 데이터 파일을 찾을 수 없습니다.')


2013년 데이터에서 널 값이 제거되었습니다.
2014년 데이터에서 널 값이 제거되었습니다.
2015년 데이터에서 널 값이 제거되었습니다.
2016년 데이터에서 널 값이 제거되었습니다.
2017년 데이터에서 널 값이 제거되었습니다.
2018년 데이터에서 널 값이 제거되었습니다.
2019년 데이터에서 널 값이 제거되었습니다.
2020년 데이터에서 널 값이 제거되었습니다.
2021년 데이터에서 널 값이 제거되었습니다.
2022년 데이터에서 널 값이 제거되었습니다.
2023년 데이터에서 널 값이 제거되었습니다.


In [11]:
player2023.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 19 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   등록번호    8 non-null      int64 
 1   이름      8 non-null      object
 2   키       8 non-null      int64 
 3   몸무게     8 non-null      int64 
 4   포지션     8 non-null      object
 5   연도      8 non-null      int64 
 6   리그      8 non-null      object
 7   소속      8 non-null      object
 8   출장      8 non-null      int64 
 9   득점      8 non-null      int64 
 10  도움      8 non-null      int64 
 11  골킥      8 non-null      int64 
 12  코너킥     8 non-null      int64 
 13  오프사이드   8 non-null      int64 
 14  슈팅      8 non-null      int64 
 15  파울      8 non-null      int64 
 16  실점      8 non-null      int64 
 17  경고      8 non-null      int64 
 18  퇴장      8 non-null      int64 
dtypes: int64(15), object(4)
memory usage: 1.3+ KB


In [6]:
import pandas as pd
import os

# 파일 경로 설정
directory = '/content/drive/MyDrive/hjh_kita_directory/hjh2차챗봇프로젝트/선수데이터셋'
start_year = 2013
end_year = 2023

# 각 파일을 읽어 DataFrame 리스트에 저장
dfs = []
for year in range(start_year, end_year + 1):
    file_path = os.path.join(directory, f'player_data_{year}.xlsx')
    df = pd.read_excel(file_path)
    dfs.append(df)

# 모든 DataFrame을 하나로 합치기
player_data = pd.concat(dfs, ignore_index=True)

# 결과 확인
player_data


,등록번호,이름,키,몸무게,포지션,연도,리그,소속,출장,득점,도움,골킥,코너킥,오프사이드,슈팅,파울,실점,경고,퇴장
0,20130026,배천석,186,78,FW,2017,K리그1,전남,8,0,1,0,0,1,3,7,0,0,0
1,20130026,배천석,186,78,FW,2016,K리그1,전남,23,3,3,0,0,4,13,12,0,3,0
2,20130026,배천석,186,78,FW,2015,K리그1,부산,21,1,1,0,0,4,30,36,0,0,0
3,20130026,배천석,186,78,FW,2014,K리그1,포항,4,0,0,0,0,1,0,5,0,0,0
4,20130026,배천석,186,78,FW,2013,K리그1,포항,20,4,2,0,0,8,19,19,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2609,20230145,카릴,185,77,FW,2023,K리그2,부천,15,2,0,0,0,2,22,10,0,0,0
2610,20230196,반토안,169,70,FW,2023,K리그2,서울E,9,0,0,0,0,1,4,7,0,1,0
2611,20230251,루안,177,70,FW,2023,K리그1,수원FC,3,0,1,0,0,0,3,2,0,0,0
2612,20230267,바카요코,178,77,FW,2023,K리그2,천안,12,0,0,0,4,2,13,4,0,0,0


In [17]:
# Excel 파일로 저장
output_file_path = '/content/drive/MyDrive/hjh_kita_directory/hjh2차챗봇프로젝트/선수데이터셋/player_data_2013-2023.xlsx'
player_data.to_excel(output_file_path, index=False)

print("파일이 성공적으로 저장되었습니다: ", output_file_path)


파일이 성공적으로 저장되었습니다:  /content/drive/MyDrive/hjh_kita_directory/hjh2차챗봇프로젝트/선수데이터셋/player_data_2013-2023.xlsx
